In [1]:
import torch
import numpy as np
import torch.nn as nn
import time

In [2]:
def load_planar_dataset(seed, m):
    np.random.seed(seed)
    m = m  # number of examples
    N = int(m / 2)  # number of points per class
    D = 2  # dimensionality
    X = np.zeros((m, D))  # data matrix where each row is a single example
    Y = np.zeros((m, 1), dtype='uint8')  # labels vector (0 for red, 1 for blue)
    a = 4  # maximum ray of the flower

    for j in range(2):
        ix = range(N * j, N * (j + 1))
        t = np.linspace(j * 3.12, (j + 1) * 3.12, N) + np.random.randn(N) * 0.2  # theta
        r = a * np.sin(4 * t) + np.random.randn(N) * 0.2  # radius
        X[ix] = np.c_[r * np.sin(t), r * np.cos(t)]
        Y[ix] = j
    return X, Y

In [3]:
class NN(torch.nn.Module):
    def __init__(self, nx, nh, ny, f1, f2):
        super(NN, self).__init__()
        # fully connected layers
        self.fc1 = torch.nn.Linear(nx, nh)
        self.fc2 = torch.nn.Linear(nh, ny)
        # functions on hid and out
        self.f1 = f1
        self.f2 = f2
        
    def forward(self, x):
        x = self.f1(self.fc1(x))
        y_pred = self.f2(self.fc2(x))
        return y_pred

In [4]:
def train(model, Xtrain, Ytrain, par, f1, f2):
#     criterion = torch.nn.BCELoss(size_average=True) # doesnt work for tanh, relu but cool for sigmoid  
    criterion = torch.nn.MSELoss(reduction='mean')
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    for epoch in range(10000):
        model.train()
        optimizer.zero_grad()    
        y_pred = model(Xtrain) # Forward pass 
        loss = criterion(y_pred, Ytrain) # Compute Loss
        loss.backward() # Backward pass
        optimizer.step() # Update bias, weights
        if epoch == 0:
            print(f"\t\tCost after 1 epoch: {loss:12.5f}")
        if epoch == 99:
            print(f"\t\tCost after 100 epochs: {loss:9.5f}")
        if epoch == 9999:
            print(f"\t\tCost after 10000 epochs: {loss:2.5f}")
    return model

In [5]:
def predict(model, Xtest, Ytest):
    y_pred = model(Xtest)
    hit_counter = 0
    for i in range(len(y_pred.data)):
        y_pred.data[i] = 1 if y_pred.data[i] > 0.5 else 0
        if y_pred.data[i] == Ytest.data[i]:
            hit_counter += 1
    print(f"\t\tAccuracy: {(hit_counter/len(y_pred.data)*100):20.2f}%")
    pass

In [6]:
# Comparison tests
fa1={"relu":torch.relu,
     "sigmoid":torch.sigmoid,
     "tanh":torch.tanh}
fa2={"relu":torch.relu,
     "sigmoid":torch.sigmoid,
     "tanh":torch.tanh}
par = {
    "nx":2,
    "nh":[1, 2, 3, 4, 5, 20, 50],
    "ny":1
}
i = 0
for f1_key, f1_val in fa1.items():
    for f2_key, f2_val in fa2.items():
        print(f"Test {i}")
        i+=1
        print(f"Activation function on hidden layer: {f1_key}\
        \nActivation function on output layer: {f2_key}")
        for n in range(len(par["nh"])):
            Xtrain, Ytrain = load_planar_dataset(1, 400) # seed = 1, trainig data amount
            Xtrain, Ytrain = torch.from_numpy(Xtrain).float(), torch.from_numpy(Ytrain).float()
            Xtest, Ytest = load_planar_dataset(2, 80) # seed = 2, test data amount
            Xtest, Ytest = torch.from_numpy(Xtest).float(), torch.from_numpy(Ytest).float()
            print(f"\tNeurons in hidden layers: {par['nh'][n]}")
            model = NN(par["nx"], par["nh"][n], par["ny"], f1_val, f2_val)
            tic = time.perf_counter()
            model = train(model, Xtrain, Ytrain, par, f1_val, f2_val)
            toc = time.perf_counter()
            print(f"\t\tTime of training: {toc - tic:13.4f} s")
            predict(model, Xtest, Ytest)

Test 0
Activation function on hidden layer: relu        
Activation function on output layer: relu
	Neurons in hidden layers: 1
		Cost after 1 epoch:      0.50000
		Cost after 100 epochs:   0.50000
		Cost after 10000 epochs: 0.50000
		Time of training:        3.0133 s
		Accuracy:                50.00%
	Neurons in hidden layers: 2
		Cost after 1 epoch:      0.33789
		Cost after 100 epochs:   0.26867
		Cost after 10000 epochs: 0.22494
		Time of training:        3.0841 s
		Accuracy:                61.25%
	Neurons in hidden layers: 3
		Cost after 1 epoch:      0.71273
		Cost after 100 epochs:   0.33634
		Cost after 10000 epochs: 0.20573
		Time of training:        3.0621 s
		Accuracy:                65.00%
	Neurons in hidden layers: 4
		Cost after 1 epoch:      0.52958
		Cost after 100 epochs:   0.27801
		Cost after 10000 epochs: 0.20255
		Time of training:        3.0104 s
		Accuracy:                68.75%
	Neurons in hidden layers: 5
		Cost after 1 epoch:      0.50017
		Cost after 100 epoc

		Cost after 10000 epochs: 0.22262
		Time of training:        3.7589 s
		Accuracy:                52.50%
	Neurons in hidden layers: 4
		Cost after 1 epoch:      0.30107
		Cost after 100 epochs:   0.28598
		Cost after 10000 epochs: 0.22182
		Time of training:        3.8599 s
		Accuracy:                50.00%
	Neurons in hidden layers: 5
		Cost after 1 epoch:      0.72805
		Cost after 100 epochs:   0.24107
		Cost after 10000 epochs: 0.14185
		Time of training:        3.8332 s
		Accuracy:                83.75%
	Neurons in hidden layers: 20
		Cost after 1 epoch:      0.83664
		Cost after 100 epochs:   0.25369
		Cost after 10000 epochs: 0.14383
		Time of training:        4.0532 s
		Accuracy:                78.75%
	Neurons in hidden layers: 50
		Cost after 1 epoch:      0.45380
		Cost after 100 epochs:   0.23897
		Cost after 10000 epochs: 0.14561
		Time of training:        5.1292 s
		Accuracy:                81.25%
Test 6
Activation function on hidden layer: tanh        
Activation function 